In [1]:
from numbers import Integral


class Person:

    @property
    def age(self):
        return getattr(self, "_age", None)

    @age.setter
    def age(self, value):
        if not isinstance(value, Integral):
            raise TypeError("age must be an integer")

        if value < 0:
            raise ValueError("age must be a positive number")

        self._age = value


In [2]:
p = Person()
try: 
    p.age = -1
except ValueError as e:
    print(e)

age must be a positive number


In [3]:
p.age = 22

In [4]:
p.__dict__

{'_age': 22}

In [5]:
class Person:

    def get_age(self):
        return getattr(self, "_age", None)

    def set_age(self, value):
        if not isinstance(value, Integral):
            raise TypeError("age must be an integer")

        if value < 0:
            raise ValueError("age must be a positive number")

        self._age = value

    age = property(fget=get_age, fset=set_age)


In [6]:
Person.age, dir(Person.age)

(<property at 0x10f7e1350>,
 ['__class__',
  '__delattr__',
  '__delete__',
  '__dir__',
  '__doc__',
  '__eq__',
  '__format__',
  '__ge__',
  '__get__',
  '__getattribute__',
  '__getstate__',
  '__gt__',
  '__hash__',
  '__init__',
  '__init_subclass__',
  '__isabstractmethod__',
  '__le__',
  '__lt__',
  '__ne__',
  '__new__',
  '__reduce__',
  '__reduce_ex__',
  '__repr__',
  '__set__',
  '__set_name__',
  '__setattr__',
  '__sizeof__',
  '__str__',
  '__subclasshook__',
  'deleter',
  'fdel',
  'fget',
  'fset',
  'getter',
  'setter'])

In [7]:
prop = Person.age
hasattr(prop, "__get__"), hasattr(prop, "__set__"), hasattr(prop, "__set_name__")

(True, True, True)

In [8]:
p = Person()
p.age = 34

In [9]:
p.age

34

In [10]:
class TimeUTC:
    @property
    def current_time(self):
        return "the time"


In [11]:
t = TimeUTC()
hasattr(TimeUTC.current_time, "__get__"), hasattr(TimeUTC.current_time, "__set__")

(True, True)

In [12]:
t.current_time

'the time'

In [13]:
try:
    # property has __set__ method, but fset function hasn't been given
    t.current_time = "other time"
except AttributeError as e:
    print(e)

property 'current_time' of 'TimeUTC' object has no setter


In [14]:
p = Person()
p.__dict__

{}

In [15]:
p.age = 10
p.age

10

In [16]:
p.__dict__  # `age` (property name) is not in the instance __dict__

{'_age': 10}

In [17]:
p.__dict__["age"] = 22

In [18]:
p.__dict__

{'_age': 10, 'age': 22}

In [19]:
p.age  # data descriptor is used instead of direct `age` key

10

In [20]:
class MakeProperty:
    def __init__(self, fget = None, fset = None):
        self.fget = fget
        self.fset = fset

    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name

    def __get__(self, instance, owner_class):
        print("__get__ called")
        if instance is None:
            print("no instance given")
            return self

        if self.fget is None:
            raise AttributeError(f"{self.property_name} is not readable.")
            
        return self.fget(instance)
    
    def __set__(self, instance, value):
        print("__set__ called")
        if self.fset is None:
            raise AttributeError(f"{self.property_name} is not writable.")

        self.fset(instance, value)


In [21]:
class Person:

    def get_name(self):
        return getattr(self, "_name", None)

    def set_name(self, value):
        if not isinstance(value, str):
            raise TypeError("name must be a string")

        self._name = value

    name = MakeProperty(fget=get_name, fset=set_name)


In [22]:
p = Person()
p.name = "Bob"

__set__ called


In [23]:
p.__dict__

{'_name': 'Bob'}

In [24]:
p.name

__get__ called


'Bob'

In [25]:
p.__dict__["name"] = "John"

In [26]:
p.__dict__

{'_name': 'Bob', 'name': 'John'}

In [27]:
p.name  # data descriptor used!

__get__ called


'Bob'

In [28]:
class Person:
    @MakeProperty
    def age(self):
        return 33


In [29]:
p = Person()
p.age

__get__ called


33

In [30]:
Person.age

__get__ called
no instance given


In [31]:
class MakeProperty:
    def __init__(self, fget = None, fset = None):
        self.fget = fget
        self.fset = fset

    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name

    def __get__(self, instance, owner_class):
        print("__get__ called")
        if instance is None:
            print("no instance given")
            return self

        if self.fget is None:
            raise AttributeError(f"{self.property_name} is not readable.")
            
        return self.fget(instance)
    
    def __set__(self, instance, value):
        print("__set__ called")
        if self.fset is None:
            raise AttributeError(f"{self.property_name} is not writable.")

        self.fset(instance, value)

    def setter(self, fset):
        # used with decorator syntax
        self.fset = fset
        return self


In [32]:
class Person:
    @MakeProperty
    def name(self):
        return getattr(self, "_name", None)

    # at this point `name` is an instance of MakeProperty, so `setter` method can be used
    @name.setter
    def name(self, value):
        self._name = value


In [33]:
p = Person()
p.name = "Bob"

__set__ called


In [34]:
p.name

__get__ called


'Bob'

In [35]:
p.__dict__

{'_name': 'Bob'}